In [ ]:
import math
import os
from typing import TYPE_CHECKING, List

import boto3
import rasterio
from rasterio.session import AWSSession
from rasterio.windows import Window

# Set environment variables for GDAL and AWS
os.environ["GDAL_HTTP_UNSAFESSL"] = "YES"
os.environ["CPL_VSIL_CURL_USE_HEAD"] = "NO"
os.environ["AWS_REQUEST_PAYER"] = "requester"

# Create AWS session
aws_session = AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
# Using a sample COG file from AWS Registry of Open Data
uri = "s3://sentinel-cogs/sentinel-s2-l2a-cogs/1/C/CV/2023/1/S2B_1CCV_20230111_0_L2A/B02.tif"

# Define number of partitions -> n_splits ** 2
n_splits = 2

In [ ]:
# List with different partitions
slices = []

# Partition logic
# Notice that programmer is responsible of program the partition logic into their code
with rasterio.Env(aws_session=aws_session):
    with rasterio.open(uri) as src:
        step_w = src.width / n_splits
        step_h = src.height / n_splits

        for bx in range(n_splits):
            for by in range(n_splits):
                off_w = round(step_w * by)
                off_h = round(step_h * bx)
                w = math.ceil(step_w * (by + 1)) - off_w
                h = math.ceil(step_h * (bx + 1)) - off_h

                window = Window(off_w, off_h, w, h)
                slices.append(window)

In [ ]:
# Slices references (not evaluated yet)
for window in slices:
    print(window)

In [ ]:
# Print the raw data of partitions
for window in slices: 
    with rasterio.Env(aws_session=aws_session):
        with rasterio.open(uri) as src:
            data = src.read(window=window)
        print(data)
        print("---------------")


```python
# Notice that slices (that are lazy evaluated) can be passed to remote
# workers (Dask, Ray, Lithops...)

import dask.bag as db

def process_window(window):
    ...

results = db.from_sequence(slices).map(process_window).compute()
```